In [11]:
!pip install polars openai


In [12]:
import marimo as mo
import re 
import os
import polars as pl
from openai import OpenAI
from tqdm import tqdm

# Setup

In [13]:

%cd students/srutiban

/home/jovyan/494-user-trajectories/students/srutiban


In [14]:
# Set your API key
# Make sure to create the file OPENAIKEY.txt before running this
# (You can use the OPENAIKEY.txt.template file as a template)
with open("secrets/OPENAIKEY.txt", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()
client = OpenAI()

In [15]:
%cd ../../

/home/jovyan/494-user-trajectories


In [16]:
# Load MITweet dataset
df = pl.read_csv("data/mitweet_sample.csv")

In [17]:
# Look at data
df

topic,tweet,partisan_lean
str,str,str
"""Black Lives Matter""","""#BlackLivesMatter ride to #Fer…","""LEFT"""
"""Women’s Right""","""Oh if you have Disney+ I reall…","""LEFT"""
"""Political Parties""","""This was no mistake. This was …","""CENTER"""
"""Abortion""","""Biden still seems to think the…","""RIGHT"""
"""Political Parties""","""Never thought I would live in …","""LEFT"""
…,…,…
"""Russo-Ukrainian War""","""'Levelling-up' is a scam. Canc…","""LEFT"""
"""Abortion""","""New: Texas can enforce its pre…","""MIXED"""
"""Russo-Ukrainian War""","""Will join and on at 9: 3…","""CENTER"""


# Labeling

In [18]:
SIMPLE_PROMPT_TEMPLATE = """# TWEET
{tweet}

# ANALYSIS INSTRUCTIONS

Use chain-of-thought reasoning to classify this tweet's partisan lean.

**Step 1: Summarize the tweet's argument**
What is this tweet claiming, advocating, or criticizing?

**Step 2: Determine direction**
Does the tweet align with popular rhetoric for either left or right partisan leanings?
Does it align with neither? Is it not political at all?

Sort tweets into LEFT, RIGHT, CENTER, MIXED, and NONE

Left-leaning can be:
- tweet speaks positively of things like DEI, BLM, LGBTQ rights, abortion, the Democratic Party
- tweet speaks negatively of things ideals of the Republican Party, Trump

Right-leaning can be:
- tweet speaks negatively of things like DEI, BLM, LGBTQ rights, abortion, the Democratic Party
- tweet speaks positively of things like the ideals of the Republic Party, Trump

Center can be:
- tweet mainly consists of information, no opinions from the user
- tweet may contain opinions in the form of quotes from other people
- tweet does not speak positively or negatively about either party


Mixed can be:
- tweet criticizes both the Democratic and Republic Party
- tweet does not seem to agree fully with either Party's rhetoric
- tweet seems anti-government

None can be:
- pop culture news, opinions about fictional characters or celebrities

# RESPONSE FORMAT

<analysis>
**Tweet's main argument:** [1-2 sentences]

**Directional assessment:** [Direction] because [1-2 sentence reason]
</analysis>

<output>
[LEFT/CENTER/RIGHT/MIXED]
</output>

Or if tweet is not political:
<output>
NONE
</output>
"""

In [19]:

def _parse_output(output_text: str) -> str:
    text = (output_text or "").strip()
    # Prefer the explicit <output> block if present
    m = re.search(r"<output>\s*(.*?)\s*</output>", text, flags=re.DOTALL | re.IGNORECASE)
    if m:
        text = m.group(1).strip()
    return text.strip().upper()

def _query_llm(row: dict) -> dict:
    tweet = row["tweet"]
    prompt = SIMPLE_PROMPT_TEMPLATE.format(tweet=tweet)
    resp = client.responses.create(model="gpt-4.1-mini", input=prompt)
    output_text = getattr(resp, "output_text", "") or ""
    return output_text


# Process rows with a for-loop
results = []
for row in tqdm(df.iter_rows(named=True), total=df.height):
    output_text = _query_llm(row)
    prediction = _parse_output(output_text)
    # Combine original row data with classification results
    result_row = {**row, **{"llm_output": output_text, "prediction": prediction}}
    results.append(result_row)

# Convert results back to a DataFrame
simple_predictions = pl.DataFrame(results)

  0%|          | 0/100 [00:00<?, ?it/s]


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************yo8A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}

# Evaluation

In [ ]:
# Look at results
pl.DataFrame(results)

In [ ]:
mo.ui.table(simple_predictions, wrap=True)


## Confusion Matrix

In [ ]:
crosstab = (
    pl.DataFrame(results)
    .group_by('partisan_lean', 'prediction')
    .len()
    .pivot(index="partisan_lean", on="prediction", values="len")
)

# Get prediction columns (everything except the index)
prediction_columns = [col for col in crosstab.columns if col != "partisan_lean"]

crosstab = (
    crosstab
    .with_columns(
        pl.concat_str([pl.lit("actually_"), pl.col("partisan_lean")]).alias("partisan_lean")
    )
    .rename({
        "partisan_lean": "actual_label",
        **{col: f"predicted_{col}" for col in prediction_columns}
    })
)

crosstab